In [1]:
import os
import sys
import csv
import arcpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Input and output file paths
csv_file_path = r'C://Users//betebari//Documents//InSAR-Subsidence//interbed_thickness_comparison.csv'
# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
print(df.head())

   NodeID            X            Y      L1      L2      L3    L4   DC_L1  \
0       1  554210.8184  4498111.367  136.39  122.35  124.57  50.0   93.97   
1       2  556163.7904  4499563.238  200.44  127.51  104.08  50.0  149.00   
2       3  557356.8226  4501930.315  203.87  137.82   82.27  50.0  149.00   
3       4  559132.7039  4503200.652  171.15  145.29   75.99  50.0  149.00   
4       5  561893.4429  4503238.309  287.87  148.00   80.04  50.0  149.00   

   DC_L2  DC_L3  DC_L4  
0   49.0   49.0  24.17  
1   49.0   49.0  24.18  
2   49.0   49.0  26.65  
3   49.0   49.0  28.78  
4   49.0   49.0  31.57  


In [3]:
# Ensure the required columns exist
required_columns = ['L1', 'DC_L1', 'L2', 'DC_L2', 'L3', 'DC_L3', 'L4', 'DC_L4']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Missing required columns: {missing_columns}")

# Calculate differences
df['Diff_L1_DC_L1'] = df['L1'] - df['DC_L1']
df['Diff_L2_DC_L2'] = df['L2'] - df['DC_L2']
df['Diff_L3_DC_L3'] = df['L3'] - df['DC_L3']
df['Diff_L4_DC_L4'] = df['L4'] - df['DC_L4']

# Save the updated DataFrame (optional)
updated_csv_path = r'C://Users//betebari//Documents//InSAR-Subsidence//Updated_Subsidence.csv'
df.to_csv(updated_csv_path, index=False)
print(f"Updated CSV saved to: {updated_csv_path}")

Updated CSV saved to: C://Users//betebari//Documents//InSAR-Subsidence//Updated_Subsidence.csv


In [16]:
# Enable the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# File paths
csv_file_path = r'C://Users//betebari//Documents//InSAR-Subsidence//Updated_Subsidence.csv'
gdb_folder = r'C://Users//betebari//Documents//InSAR-Subsidence'
gdb_name = "Subsidence.gdb"
gdb_path = os.path.join(gdb_folder, gdb_name)
mask_shapefile = r"C:/Users/betebari/Documents/C2VSim_Texture/OSWCR/central_val_buf_5mil_utm10n.shp"

# Ensure the geodatabase exists
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(gdb_folder, gdb_name)

# Load the DataFrame
df = pd.read_csv(csv_file_path)

# Columns to interpolate
columns_to_interpolate = ['Diff_L1_DC_L1', 'Diff_L2_DC_L2', 'Diff_L3_DC_L3', 'Diff_L4_DC_L4']

# Spatial reference
spatial_ref = arcpy.SpatialReference(26910)  # NAD83 UTM Zone 10N

# Set the ArcPy environment
arcpy.env.workspace = gdb_path
arcpy.env.outputCoordinateSystem = spatial_ref
arcpy.env.extent = mask_shapefile  # Restrict extent to the shapefile
arcpy.env.mask = mask_shapefile    # Restrict interpolation to the shapefile

# Create a temporary point feature class in the geodatabase
points_fc = f"{gdb_path}/TempPoints"
if arcpy.Exists(points_fc):
    arcpy.Delete_management(points_fc)

arcpy.CreateFeatureclass_management(gdb_path, "TempPoints", "POINT", spatial_reference=spatial_ref)

# Add fields for each column to interpolate
for col in columns_to_interpolate:
    arcpy.AddField_management(points_fc, col, "DOUBLE")

# Insert points into the feature class
with arcpy.da.InsertCursor(points_fc, ["SHAPE@XY"] + columns_to_interpolate) as cursor:
    for _, row in df.iterrows():
        cursor.insertRow([(row['X'], row['Y'])] + [row[col] for col in columns_to_interpolate])

# Interpolate using IDW for each column
for col in columns_to_interpolate:
    raster_output = f"{gdb_path}/{col}_IDW"
    idw_result = arcpy.sa.Idw(
        in_point_features=points_fc,
        z_field=col,
        cell_size=100,  # Adjust cell size as needed
        power=2  # Exponent for IDW; higher values give more weight to closer points
    )
    idw_result.save(raster_output)
    print(f"IDW Raster created: {raster_output}")

# Clean up temporary feature class
arcpy.Delete_management(points_fc)

# Release the Spatial Analyst extension
arcpy.CheckInExtension("Spatial")

IDW Raster created: C://Users//betebari//Documents//InSAR-Subsidence\Subsidence2.gdb/Diff_L1_DC_L1_IDW
IDW Raster created: C://Users//betebari//Documents//InSAR-Subsidence\Subsidence2.gdb/Diff_L2_DC_L2_IDW
IDW Raster created: C://Users//betebari//Documents//InSAR-Subsidence\Subsidence2.gdb/Diff_L3_DC_L3_IDW
IDW Raster created: C://Users//betebari//Documents//InSAR-Subsidence\Subsidence2.gdb/Diff_L4_DC_L4_IDW


'CheckedIn'